In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all convolutional layers (we only train the new layers)
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Ensure x is built from the base_model's output
x = base_model.output  
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Now create the model
model = Model(inputs=base_model.input, outputs=x)


In [4]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from dataset
train_data = train_datagen.flow_from_directory(
    "/kaggle/input/hot-dog-not-hot-dog/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    "/kaggle/input/hot-dog-not-hot-dog/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [5]:
print(train_data.class_indices)


{'hot_dog': 0, 'not_hot_dog': 1}


In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
model.fit(train_data, validation_data=val_data, epochs=25)

# Save the trained model
model.save("hotdog_not_hotdog.h5")


Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.4603 - loss: 2.5374 - val_accuracy: 0.6540 - val_loss: 0.6201
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 310ms/step - accuracy: 0.5232 - loss: 0.9205 - val_accuracy: 0.6820 - val_loss: 0.5833
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 318ms/step - accuracy: 0.6299 - loss: 0.7297 - val_accuracy: 0.6940 - val_loss: 0.6145
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 304ms/step - accuracy: 0.7252 - loss: 0.5441 - val_accuracy: 0.7800 - val_loss: 0.4514
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 323ms/step - accuracy: 0.7611 - loss: 0.4627 - val_accuracy: 0.7700 - val_loss: 0.5008
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 313ms/step - accuracy: 0.7723 - loss: 0.4495 - val_accuracy: 0.7940 - val_loss: 0.4435
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 305ms/step - accuracy: 0.8284 - loss: 0.3489 - val_accuracy: 0.7900 - val_loss: 0.4574
Epoch 8/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 288ms/step - accuracy: 0.8650 - loss: 0.3255 - val_accuracy: 0.7500 - val_lo